In [2]:
import numpy as np
import decimal

In [1]:
def Pivoting(A, b, s, n, k):
# A: Coef. of matrix A; 2-D array
# b: Coef. of vector b; 1-D array it is None if i didn't pass one
# n: Dimension of the system of equations
# s: n-element array for storing scaling factors

  p = k

  if s is not None:

    # Finding the largest scaled coefficient in column k
    big = abs(A[k][k] / s[k])
    for i in range(k+1,n) :
      dummy = abs(A[i][k] / s[i]) # dummy number for the scaled value
      if dummy > big:
        big = dummy
        p = i # new pivoting row

    # Next: Swap row p and row k if p != k
    if p != k:
      # Swap row p and row k
      for j in range(k,n) :
        dummy = A[p][j]
        A[p][j] = A[k][j]
        A[k][j] = dummy
      if b is not None:
        dummy = b[p]
        b[p] = b[k]
        b[k] = dummy

      dummy = s[p]
      s[p] = s[k]
      s[k] = dummy
  else:
  # Finding the largest coefficient in column k
    big = abs(A[k][k])
    for i in range(k+1,n) :
      dummy = abs(A[i][k] ) # dummy number for the  value
      if dummy > big:
        big = dummy
        p = i # new pivoting row

    # Swap row p and row k if p != k
    if p != k:
      for j in range(k,n) :
        dummy = A[p][j]
        A[p][j] = A[k][j]
        A[k][j] = dummy
      if b is not None:
        dummy = b[p]
        b[p] = b[k]
        b[k] = dummy


In [3]:
def back_substitution(A, b, n):
  x = [0.0 for _ in range(n)]
  x[n-1] = b[n-1] / A[n-1][n-1]
  for i in range(n-2,-1,-1):
    sum_val = 0.0
    for j in range(i+1,n):
      sum_val = sum_val + A[i][j] * x[j]
    x[i] = (b[i] - sum_val) / A[i][i]
  return x


In [ ]:
def forward_elimination(A, b, s, n, tol):
  """
  A: Coef. of matrix A; 2-D array
  b: Coef. of vector b; 1-D array it is None if i didn't pass one
  n: Dimension of the system of equations
  tol: Tolerance; smallest possible scaled pivot allowed
  s: n-element array for storing scaling factors
  """

  if s is not None and b is not None:
    for k  in range(n-1):
      Pivoting(A, b, s, n, k) # Partial Pivoting
      if abs(A[k][k] / s[k]) < tol:  # Check for singularity
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]
        b[i] = b[i] - factor * b[k]

    if abs(A[n-1][n-1]/s[n-1]) < tol: # Check for singularity
      return -1
  elif s is None and b is not None: # no scaling
    for k  in range(n-1):
      Pivoting(A, b, None, n, k) # Partial Pivoting
      if abs(A[k][k]) < tol:
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]
        b[i] = b[i] - factor * b[k]

    if abs(A[n-1][n-1]) < tol: # Check for singularity
      return -1
  elif s is not None and b is None:
    for k  in range(n-1):
      Pivoting(A, None, s, n, k) # Partial Pivoting
      if abs(A[k][k] / s[k]) < tol:  # Check for singularity
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]

    if abs(A[n-1][n-1]/s[n-1]) < tol: # Check for singularity
      return -1
  else: # no scaling, b is None
    for k  in range(n-1):
      Pivoting(A, None, None, n, k) # Partial Pivoting
      if abs(A[k][k]) < tol:
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]

    if abs(A[n-1][n-1]) < tol: # Check for singularity
      return -1
  return 0 # Return 0 for successful elimination
     

In [8]:
def backward_elimination(A, b, s, n, tol):
    """  
    Parameters:
    A: Coefficient matrix (upper triangular after forward elimination); 2-D array
    b: Right-hand side vector; 1-D array (can be None)
    s: Scaling factors array; 1-D array (can be None)
    n: Dimension of the system
    tol: Tolerance for singularity check
    """
    
    if s is not None and b is not None:
        for k in range(n-1, 0, -1):  # Start from last row, go up to row 1
            if abs(A[k][k] / s[k]) < tol:  # Check for singularity
                return -1
            for i in range(k-1, -1, -1):  # Eliminate above pivot
                factor = A[i][k] / A[k][k]
                A[i][k] = 0.0  # Will become zero
                b[i] = b[i] - factor * b[k]
        
        # Normalize diagonal to 1 and scale solution
        for i in range(n):
            if abs(A[i][i] / s[i]) < tol:
                return -1
            b[i] = b[i] / A[i][i]
            A[i][i] = 1.0
            
    elif s is None and b is not None:  # No scaling
        for k in range(n-1, 0, -1):
            if abs(A[k][k]) < tol:  # Check for singularity
                return -1
            for i in range(k-1, -1, -1):
                factor = A[i][k] / A[k][k]
                A[i][k] = 0.0
                b[i] = b[i] - factor * b[k]
        
        for i in range(n):
            if abs(A[i][i]) < tol:
                return -1
            b[i] = b[i] / A[i][i]
            A[i][i] = 1.0
            
    elif s is not None and b is None:  # With scaling, no b vector
        for k in range(n-1, 0, -1):
            if abs(A[k][k] / s[k]) < tol:
                return -1
            for i in range(k-1, -1, -1):
                factor = A[i][k] / A[k][k]
                A[i][k] = 0.0
        
        for i in range(n):
            if abs(A[i][i] / s[i]) < tol:
                return -1
            A[i][i] = 1.0
            
    else:  # No scaling, no b vector
        for k in range(n-1, 0, -1):
            if abs(A[k][k]) < tol:
                return -1
            for i in range(k-1, -1, -1):
                factor = A[i][k] / A[k][k]
                A[i][k] = 0.0
        
        for i in range(n):
            if abs(A[i][i]) < tol:
                return -1
            A[i][i] = 1.0
    
    return 0  # Success


In [9]:
def gauss_jordan(A, b, s, n, tol):
    """
    Parameters:
    A: Coefficient matrix; 2-D array
    b: Right-hand side vector; 1-D array (can be None)
    s: Scaling factors array; 1-D array (can be None for no scaling)
    n: Dimension of the system
    tol: Tolerance for singularity check
    """
    
    # Forward elimination
    result = forward_elimination(A, b, s, n, tol)
    if result == -1:
        return None  # Singular or ill-conditioned matrix
    
    # Backward elimination
    result = backward_elimination(A, b, s, n, tol)
    if result == -1:
        return None
    
    return b  # Solution is now in b vector

In [ ]:
import numpy as np
import copy

# Test Case 1: Simple 3x3 system with scaling
print("="*60)
print("Test Case 1: Simple 3x3 system with scaling")
print("="*60)
A1 = np.array([[2.0, 1.0, -1.0], 
               [-3.0, -1.0, 2.0], 
               [-2.0, 1.0, 2.0]], dtype=float)
b1 = np.array([8.0, -11.0, -3.0], dtype=float)
n1 = 3
tol1 = 1e-12

# Calculate scaling factors
s1 = np.array([np.max(np.abs(A1[i, :])) for i in range(n1)])

# Make copies
A1_copy = copy.deepcopy(A1)
b1_copy = copy.deepcopy(b1)

print("Original A:")
print(A1)
print("\nOriginal b:")
print(b1)
print("\nScaling factors:", s1)

# Solve with Gauss-Jordan
solution1 = gauss_jordan(A1_copy, b1_copy, s1, n1, tol1)

if solution1 is not None:
    print("\nSolution:", solution1)
    # Verify solution
    residual = np.dot(A1, solution1) - b1
    print("Residual (Ax - b):", residual)
    print("Max error:", np.max(np.abs(residual)))
else:
    print("\nSystem is singular")

# Test Case 2: 3x3 system without scaling
print("\n" + "="*60)
print("Test Case 2: Same system without scaling")
print("="*60)
A2_copy = copy.deepcopy(A1)
b2_copy = copy.deepcopy(b1)

solution2 = gauss_jordan(A2_copy, b2_copy, None, n1, tol1)

if solution2 is not None:
    print("Solution:", solution2)
    residual = np.dot(A1, solution2) - b1
    print("Residual (Ax - b):", residual)
    print("Max error:", np.max(np.abs(residual)))
else:
    print("System is singular")

# Test Case 3: 4x4 system with scaling
print("\n" + "="*60)
print("Test Case 3: 4x4 system with scaling")
print("="*60)
A3 = np.array([[4.0, -1.0, 0.0, 0.0],
               [-1.0, 4.0, -1.0, 0.0],
               [0.0, -1.0, 4.0, -1.0],
               [0.0, 0.0, -1.0, 4.0]], dtype=float)
b3 = np.array([15.0, 10.0, 10.0, 10.0], dtype=float)
n3 = 4
tol3 = 1e-12

s3 = np.array([np.max(np.abs(A3[i, :])) for i in range(n3)])

A3_copy = copy.deepcopy(A3)
b3_copy = copy.deepcopy(b3)

print("Original A:")
print(A3)
print("\nOriginal b:")
print(b3)

solution3 = gauss_jordan(A3_copy, b3_copy, s3, n3, tol3)

if solution3 is not None:
    print("\nSolution:", solution3)
    residual = np.dot(A3, solution3) - b3
    print("Residual (Ax - b):", residual)
    print("Max error:", np.max(np.abs(residual)))
else:
    print("\nSystem is singular")

# Test Case 4: Ill-conditioned system (large condition number)
print("\n" + "="*60)
print("Test Case 4: Ill-conditioned system")
print("="*60)
A4 = np.array([[1.0, 2.0, 3.0],
               [2.0, 4.0, 6.0],
               [1.0, 2.0, 4.0]], dtype=float)
b4 = np.array([6.0, 12.0, 7.0], dtype=float)
n4 = 3
tol4 = 1e-12

s4 = np.array([np.max(np.abs(A4[i, :])) for i in range(n4)])

A4_copy = copy.deepcopy(A4)
b4_copy = copy.deepcopy(b4)

print("Original A (should be nearly singular):")
print(A4)
print("\nOriginal b:")
print(b4)

solution4 = gauss_jordan(A4_copy, b4_copy, s4, n4, tol4)

if solution4 is not None:
    print("\nSolution:", solution4)
else:
    print("\nSystem is singular (as expected)")

# Test Case 5: Identity-like system
print("\n" + "="*60)
print("Test Case 5: Identity-like system")
print("="*60)
A5 = np.array([[5.0, 0.0, 0.0],
               [0.0, 3.0, 0.0],
               [0.0, 0.0, 2.0]], dtype=float)
b5 = np.array([10.0, 9.0, 8.0], dtype=float)
n5 = 3
tol5 = 1e-12

s5 = np.array([np.max(np.abs(A5[i, :])) for i in range(n5)])

A5_copy = copy.deepcopy(A5)
b5_copy = copy.deepcopy(b5)

print("Original A (diagonal):")
print(A5)
print("\nOriginal b:")
print(b5)

solution5 = gauss_jordan(A5_copy, b5_copy, s5, n5, tol5)

if solution5 is not None:
    print("\nSolution:", solution5)
    print("Expected: [2.0, 3.0, 4.0]")
    residual = np.dot(A5, solution5) - b5
    print("Residual (Ax - b):", residual)
else:
    print("\nSystem is singular")

# Test Case 6: Larger system (5x5)
print("\n" + "="*60)
print("Test Case 6: 5x5 Hilbert-like system")
print("="*60)
A6 = np.array([[1.0, 0.5, 0.33, 0.25, 0.2],
               [0.5, 0.33, 0.25, 0.2, 0.17],
               [0.33, 0.25, 0.2, 0.17, 0.14],
               [0.25, 0.2, 0.17, 0.14, 0.125],
               [0.2, 0.17, 0.14, 0.125, 0.11]], dtype=float)
b6 = np.array([1.0, 1.0, 1.0, 1.0, 1.0], dtype=float)
n6 = 5
tol6 = 1e-12

s6 = np.array([np.max(np.abs(A6[i, :])) for i in range(n6)])

A6_copy = copy.deepcopy(A6)
b6_copy = copy.deepcopy(b6)

print("Original A (Hilbert-like):")
print(A6)
print("\nOriginal b:")
print(b6)

solution6 = gauss_jordan(A6_copy, b6_copy, s6, n6, tol6)

if solution6 is not None:
    print("\nSolution:", solution6)
    residual = np.dot(A6, solution6) - b6
    print("Residual (Ax - b):", residual)
    print("Max error:", np.max(np.abs(residual)))
else:
    print("\nSystem is singular")

# Test Case 7: Compare with numpy's solution
print("\n" + "="*60)
print("Test Case 7: Comparison with NumPy's solution")
print("="*60)
A7 = np.array([[3.0, 2.0, -1.0],
               [2.0, -2.0, 4.0],
               [-1.0, 0.5, -1.0]], dtype=float)
b7 = np.array([1.0, -2.0, 0.0], dtype=float)
n7 = 3
tol7 = 1e-12

s7 = np.array([np.max(np.abs(A7[i, :])) for i in range(n7)])

A7_copy = copy.deepcopy(A7)
b7_copy = copy.deepcopy(b7)

print("Original A:")
print(A7)
print("\nOriginal b:")
print(b7)

solution7 = gauss_jordan(A7_copy, b7_copy, s7, n7, tol7)
numpy_solution = np.linalg.solve(A7, b7)

if solution7 is not None:
    print("\nOur solution:", solution7)
    print("NumPy solution:", numpy_solution)
    print("Difference:", np.abs(solution7 - numpy_solution))
    print("Max difference:", np.max(np.abs(solution7 - numpy_solution)))
else:
    print("\nSystem is singular")

Test Case 1: Simple 3x3 system with scaling
Original A:
[[ 2.  1. -1.]
 [-3. -1.  2.]
 [-2.  1.  2.]]

Original b:
[  8. -11.  -3.]

Scaling factors: [2. 3. 2.]

Solution: [ 2.  3. -1.]
Residual (Ax - b): [0. 0. 0.]
Max error: 0.0

Test Case 2: Same system without scaling
Solution: [ 2.  3. -1.]
Residual (Ax - b): [0.0000000e+00 0.0000000e+00 8.8817842e-16]
Max error: 8.881784197001252e-16

Test Case 3: 4x4 system with scaling
Original A:
[[ 4. -1.  0.  0.]
 [-1.  4. -1.  0.]
 [ 0. -1.  4. -1.]
 [ 0.  0. -1.  4.]]

Original b:
[15. 10. 10. 10.]

Solution: [4.97607656 4.90430622 4.64114833 3.66028708]
Residual (Ax - b): [ 0.00000000e+00  0.00000000e+00 -1.77635684e-15  0.00000000e+00]
Max error: 1.7763568394002505e-15

Test Case 4: Ill-conditioned system
Original A (should be nearly singular):
[[1. 2. 3.]
 [2. 4. 6.]
 [1. 2. 4.]]

Original b:
[ 6. 12.  7.]

System is singular (as expected)

Test Case 5: Identity-like system
Original A (diagonal):
[[5. 0. 0.]
 [0. 3. 0.]
 [0. 0. 2.]]

Or